.mp3 to .wav

In [ ]:
%pip install pydub

In [19]:
from pydub import AudioSegment

In [6]:
DATA_PATH = "./"
audio_file_name = "NceS - Burn"

In [20]:
# mp3 to wav
audio = AudioSegment.from_mp3(DATA_PATH + audio_file_name + ".mp3")
audio = audio.set_channels(1)
audio.export(DATA_PATH + audio_file_name + ".wav", format = "wav")

<_io.BufferedRandom name='./NceS - Burn.wav'>

MERT-v1-330M

In [1]:
# from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2FeatureExtractor
from transformers import AutoModel
import torch
from torch import nn
import torchaudio
import torchaudio.transforms as T


In [2]:
PRE_MODEL_PATH = "/share/LLMs/"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
# loading our model weights
model = AutoModel.from_pretrained(PRE_MODEL_PATH + "MERT-v1-330M/", trust_remote_code = True).to(DEVICE)
# loading the corresponding preprocessor config
processor = Wav2Vec2FeatureExtractor.from_pretrained(PRE_MODEL_PATH + "MERT-v1-330M/",trust_remote_code = True)

print("load model and processor successfully")

Some weights of the model checkpoint at /share/LLMs/MERT-v1-330M/ were not used when initializing MERTModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing MERTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MERTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MERTModel were not initialized from the model checkpoint at /share/LLMs/MERT-v1-330M/ and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

load model and processor successfully


In [21]:

# load demo audio and set processor
# dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
# dataset = dataset.sort("id")
orig_input_audio, sampling_rate = torchaudio.load(DATA_PATH + audio_file_name + ".wav", normalize = True)
resample_rate = processor.sampling_rate

# make sure the sample_rate aligned
if resample_rate != sampling_rate :
    print(f'setting rate from {sampling_rate} to {resample_rate}')
    resampler = T.Resample(sampling_rate, resample_rate)
else :
    resampler = None

# audio file is decoded on the fly
if not(resampler is None):
    input_audio = resampler(orig_input_audio)

setting rate from 44100 to 24000


In [24]:
inputs = processor(*input_audio, sampling_rate = resample_rate, return_tensors = "pt").to(DEVICE)

In [26]:
with torch.no_grad():
    outputs = model(**inputs, output_hidden_states = True)

print(outputs)

# take a look at the output shape, there are 25 layers of representation
# each layer performs differently in different downstream tasks, you should choose empirically
all_layer_hidden_states = torch.stack(outputs.hidden_states).squeeze()
print(all_layer_hidden_states.shape) # [25 layer, Time steps, 1024 feature_dim]

# for utterance level classification tasks, you can simply reduce the representation in time
time_reduced_hidden_states = all_layer_hidden_states.mean(-2)
print(time_reduced_hidden_states.shape) # [25, 1024]



BaseModelOutput(last_hidden_state=tensor([[[-0.0508, -0.1875,  0.2820,  ...,  0.0117, -0.0832, -0.0803],
         [-0.0121, -0.0935,  0.3626,  ...,  0.0246, -0.1199, -0.2340],
         [ 0.0147, -0.0659,  0.3629,  ..., -0.0214, -0.1301, -0.2980],
         ...,
         [ 0.0850,  0.0865,  0.3297,  ...,  0.2297, -0.0747, -0.2903],
         [ 0.0463,  0.0428,  0.1600,  ...,  0.0796, -0.0757, -0.2254],
         [ 0.1221, -0.1604,  0.1703,  ...,  0.0602, -0.0211, -0.0823]]],
       device='cuda:0'), hidden_states=(tensor([[[-0.0680,  1.3246,  0.7629,  ..., -1.3813,  3.7369,  5.7130],
         [-0.0417,  1.3526,  0.1504,  ..., -1.3390,  3.7320,  5.9386],
         [-0.0543,  1.3437, -0.0926,  ..., -1.2024,  3.7227,  5.6759],
         ...,
         [-0.4361,  2.1325, -0.1809,  ...,  2.4619,  3.5750,  6.2905],
         [ 0.5461,  1.8467, -0.2630,  ...,  1.6544,  3.0875,  5.4737],
         [-0.1295,  1.2432, -0.1485,  ...,  1.4961,  3.6980,  5.2830]]],
       device='cuda:0'), tensor([[[-1.3213